In [1]:
!pip install --upgrade datasets fsspec

  Using cached fsspec-2025.5.1-py3-none-any.whl.metadata (11 kB)


In [4]:
import pandas as pd
from tqdm import tqdm
from datasets import load_dataset
from transformers import AutoModel, AutoTokenizer
from sklearn.metrics.pairwise import cosine_similarity
import torch
from datasets import Dataset
import requests
import io



In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [6]:
model_name="bigscience/bloom-1b7"
tokenizer=AutoTokenizer.from_pretrained(model_name)
model=AutoModel.from_pretrained(model_name).to(device)
# Embedding function
def get_embedding(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    inputs = {k: v.to(device) for k, v in inputs.items()}

    with torch.no_grad():
        outputs = model(**inputs)

    token_embeddings = outputs.last_hidden_state.squeeze(0)  # shape: [seq_len, hidden_dim]
    input_ids = inputs["input_ids"].squeeze(0)

    special_token_ids = {
        tokenizer.pad_token_id,
        tokenizer.cls_token_id,
        tokenizer.sep_token_id,
        tokenizer.bos_token_id,
        tokenizer.eos_token_id
    }

    mask = torch.tensor([token_id.item() not in special_token_ids for token_id in input_ids], device=device)
    filtered_embeddings = token_embeddings[mask]
    mean_embedding = filtered_embeddings.mean(dim=0)

    return mean_embedding.cpu().numpy()model.eval()

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


BloomModel(
  (word_embeddings): Embedding(250880, 2048)
  (word_embeddings_layernorm): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
  (h): ModuleList(
    (0-23): 24 x BloomBlock(
      (input_layernorm): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
      (self_attention): BloomAttention(
        (query_key_value): Linear(in_features=2048, out_features=6144, bias=True)
        (dense): Linear(in_features=2048, out_features=2048, bias=True)
        (attention_dropout): Dropout(p=0.0, inplace=False)
      )
      (post_attention_layernorm): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
      (mlp): BloomMLP(
        (dense_h_to_4h): Linear(in_features=2048, out_features=8192, bias=True)
        (gelu_impl): BloomGelu()
        (dense_4h_to_h): Linear(in_features=8192, out_features=2048, bias=True)
      )
    )
  )
  (ln_f): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
)

In [7]:
# Loading lexicon dataset available on Hugging Face
dataset = load_dataset("MLap/English-French-Portuguese-Lexicon")

df = dataset["train"].to_pandas().dropna()


README.md:   0%|          | 0.00/669 [00:00<?, ?B/s]

train.csv:   0%|          | 0.00/10.9k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/562 [00:00<?, ? examples/s]

In [16]:
# Embedding function
def get_embedding(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    inputs = {k: v.to(device) for k, v in inputs.items()}

    with torch.no_grad():
        outputs = model(**inputs)

    token_embeddings = outputs.last_hidden_state.squeeze(0)  # shape: [seq_len, hidden_dim]
    input_ids = inputs["input_ids"].squeeze(0)

    special_token_ids = {
        tokenizer.pad_token_id,
        tokenizer.cls_token_id,
        tokenizer.sep_token_id,
        tokenizer.bos_token_id,
        tokenizer.eos_token_id
    }

    mask = torch.tensor([token_id.item() not in special_token_ids for token_id in input_ids], device=device)
    filtered_embeddings = token_embeddings[mask]
    mean_embedding = filtered_embeddings.mean(dim=0)

    return mean_embedding.cpu().numpy()

In [18]:
# Compute similarities
results = []
for row in tqdm(df.itertuples(index=False), total=len(df)):
    en, fr, pt = row.English, row.French, row.Portuguese
    try:
        emb_en = get_embedding(en)
        emb_fr = get_embedding(fr)
        emb_pt = get_embedding(pt)

        results.append({
            "English": en,
            "French": fr,
            "Portuguese": pt,
            "sim_en_fr": cosine_similarity([emb_en], [emb_fr])[0][0],
            "sim_fr_pt": cosine_similarity([emb_fr], [emb_pt])[0][0],
            "sim_pt_en": cosine_similarity([emb_pt], [emb_en])[0][0],
        })
    except Exception as e:
        print(f"Error on row: {row} -> {e}")

100%|██████████| 562/562 [00:39<00:00, 14.12it/s]


In [19]:
# Save results to CSV
output_df = pd.DataFrame(results)
output_df.to_csv("lexicon_similarity_results.csv", index=False)

# Print average similarities
print("\nAverage Cosine Similarity:")
print(f"English–French: {output_df['sim_en_fr'].mean():.4f}")
print(f"French–Portuguese: {output_df['sim_fr_pt'].mean():.4f}")
print(f"Portuguese–English: {output_df['sim_pt_en'].mean():.4f}")

print("\nSaved to: lexicon_similarity_results.csv")


Average Cosine Similarity:
English–French: 0.9365
French–Portuguese: 0.9349
Portuguese–English: 0.9153

Saved to: lexicon_similarity_results.csv
